# Imports

In [1]:
#In order to use sonnet 1, we need to downgrade a bunch of stuff
#Using tensorflow-gpu>2 cause sonnet not finding the tensorflow-probability module. 
#There're some relative discussions online.
#!pip install "dm-sonnet<2" "tensorflow-probability==0.8.0" "tensorflow-gpu<2"
#The command cause following execution using tensorflow1
!pip install "dm-sonnet<2" "tensorflow-probability==0.8.0" "tensorflow-gpu<2"

In [3]:
import tensorflow as tf
import sonnet as snt

print("TensorFlow version {}".format(tf.__version__))
print("Sonnet version {}".format(snt.__version__))

import collections
import copy
import random
import time
import itertools

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

tf.disable_eager_execution()

TensorFlow version 1.15.4
Sonnet version 1.34


# Introduction

This colab builds a graph neural network that learns from the spatio-temporal signals (ST-GNN). 

* Model spec:
  * Both the nodes and edges features are temporal, and encoder will perform sequence compression on the temporal information.
  * Message passing unit in the ST-GNN can perform one way proprogation of neighorbing features together with the edge features.
  * Then node level aggregation is done to get the final node representation that encodes both temporal and spatial information. 
  * The ST-GNN can be extended by making modifications on the encoder, message passing, aggregation modules. For examples, add attention layer to encoder, add attention in aggregation layer.



# Build a ST-GNN model

## Graph encoder module

In [4]:
class GraphEncoder(snt.AbstractModule):
  """Module that encodes node and edge features to embedding vectors."""

  def __init__(self,
               node_hidden_sizes=None,
               edge_hidden_sizes=None,
               name='graph_encoder'):
    """Constructor.

    Args:
      node_hidden_sizes: if provided should be a list of ints, hidden sizes of
        node encoder network, the last element is the size of the node outputs.
        If not provided, node features will pass through as is.
      edge_hidden_sizes: if provided should be a list of ints, hidden sizes of
        edge encoder network, the last element is the size of the edge outptus.
        If not provided, edge features will pass through as is.
      name: name of this module.
    """
    super(GraphEncoder, self).__init__(name=name)
    self._node_hidden_sizes = node_hidden_sizes
    self._edge_hidden_sizes = edge_hidden_sizes

  def _build(self, node_features, edge_features=None):
    """Encodes the node and edge features.

    Args:
      node_features: [n_nodes, node_seq_len, node_feat_dim] float tensor.
      edge_features: if provided, should be 
        [n_edges, edge_seq_len, edge_feat_dim] float tensor.

    Returns:
      node_outputs: [n_nodes, node_embedding_dim] float tensor, node embeddings.
      edge_outputs: if both edge_features and edge_hidden_sizes is provided,
        this is [n_edges, edge_embedding_dim] float tensor, edge embeddings; 
        otherwise just the input edge_features.
    """
    if self._node_hidden_sizes is None:
      node_outputs = node_features
    else:
      nodel_lstm_layers = []
      for hidden_size in self._node_hidden_sizes:
        nodel_lstm_layers.append(snt.LSTM(hidden_size))
      node_lstm = snt.DeepRNN(nodel_lstm_layers, 
                              skip_connections=True, 
                              name='node_deep_rnn')
      node_features_t = tf.transpose(node_features, perm=[1, 0, 2]) 
      node_outputs, node_final_state = tf.nn.dynamic_rnn(
          node_lstm, node_features_t, dtype=tf.float32)

    if edge_features is None or self._edge_hidden_sizes is None:
      edge_outputs = edge_features
    else:
      edge_lstm_layers = []
      for hidden_size in self._edge_hidden_sizes:
        edge_lstm_layers.append(snt.LSTM(hidden_size))
      edge_lstm = snt.DeepRNN(edge_lstm_layers, 
                              skip_connections=True, name='edge_deep_rnn')      
      edge_features_t = tf.transpose(edge_features, perm=[1, 0, 2]) 
      edge_outputs, edge_final_state = tf.nn.dynamic_rnn(
          edge_lstm, edge_features_t, dtype=tf.float32) 

    # Use the last hiddent state  
    return node_outputs[-1], edge_outputs[-1]

## Graph message passing module

In [5]:
def graph_prop_once(node_states,
                    from_idx,
                    to_idx,
                    message_net,
                    aggregation_module=tf.math.unsorted_segment_mean,
                    edge_features=None):
  """Performs one round of propagation (message passing) in a graph. 
    Updates the messags(at edge level) and aggregate the messages at node level.

  Args:
    node_states: [n_nodes, node_state_dim] float tensor, one row for each node.
    from_idx: [n_edges] int tensor, index of the from nodes.
    to_idx: [n_edges] int tensor, index of the to nodes.
    message_net: a network that maps concatenated edge inputs to message vectors.
    aggregation_module: a module that aggregates messages on edges to aggregated
      messages for each node. Should be a callable and can be called like the
      following,
        `aggregated_messages = aggregation_module(messages, to_idx, n_nodes)`,
      where messages is [n_edges, edge_message_dim] tensor, to_idx is the index
      of the to_nodes, i.e. where each message goes to, and n_nodes is an
      int which is the number of nodes to aggregate into.
    edge_features: if provided, should be a [n_edges, edge_feature_dim] float
      tensor, extra features for each edge.

  Returns:
    aggregated_messages: [n_nodes, edge_message_dim] float tensor, the
      aggregated messages, one row for each node.
  """
  from_states = tf.gather(node_states, from_idx)
  to_states = tf.gather(node_states, to_idx)

  edge_inputs = [from_states, to_states]
  if edge_features is not None:
    edge_inputs.append(edge_features)

  edge_inputs = tf.concat(edge_inputs, axis=-1)
  messages = message_net(edge_inputs)

  return aggregation_module(messages, to_idx, tf.shape(node_states)[0]), messages


class GraphPropLayer(snt.AbstractModule):
  """Graph propagation (or message passing) module."""

  def __init__(self,
               node_state_dim,
               edge_hidden_sizes,
               node_hidden_sizes,
               edge_net_init_scale=0.1,
               node_update_type='gru',
               use_reverse_direction=False,
               reverse_dir_param_different=True,
               aggregation_module=tf.math.unsorted_segment_mean,
               layer_norm=True,
               name='graph_mp'):
    """Constructor.

    Args:
      node_state_dim: int, the dimension of node states.
      edge_hidden_sizes: list of ints, hidden sizes for the edge message
        net, the last element in the list is the size of the message vectors.
      node_hidden_sizes: list of ints, hidden sizes for the node update
        net.
      edge_net_init_scale: initialization scale for the edge networks.  This
        is typically set to a small value such that the gradient does not blow
        up.
      node_update_type: type of node updates, one of {mlp, gru, residual}.
      use_reverse_direction: set to True to also propagate messages in the
        reverse direction.
      reverse_dir_param_different: set to True to have the messages computed
        using a different set of parameters than for the forward direction.
      layer_norm: set to True to use layer normalization.
      name: name of this module.
    """
    super(GraphPropLayer, self).__init__(name=name)

    self._node_state_dim = node_state_dim
    self._edge_hidden_sizes = edge_hidden_sizes[:]

    # output size is node_state_dim
    self._node_hidden_sizes = node_hidden_sizes[:] + [node_state_dim]
    self._edge_net_init_scale = edge_net_init_scale
    self._node_update_type = node_update_type
    self._aggregation_module = aggregation_module

    self._use_reverse_direction = use_reverse_direction
    self._reverse_dir_param_different = reverse_dir_param_different

    self._layer_norm = layer_norm

  def _compute_aggregated_messages(
      self, node_states, from_idx, to_idx, edge_features=None):
    """Computes aggregated messages for each node.

    Args:
      node_states: [n_nodes, input_node_state_dim] float tensor, node states.
      from_idx: [n_edges] int tensor, from node indices for each edge.
      to_idx: [n_edges] int tensor, to node indices for each edge.
      edge_features: if not None, should be [n_edges, edge_embedding_dim]
        tensor, edge features.

    Returns:
      aggregated_messages: [n_nodes, aggregated_message_dim] float tensor, the
        aggregated messages for each node.
    """
    self._message_net = snt.nets.MLP(
        self._edge_hidden_sizes,
        initializers={
            'w': tf.variance_scaling_initializer(
                scale=self._edge_net_init_scale),
            'b': tf.zeros_initializer()},
        name='message_mlp')

    aggregated_messages, new_edge_features = graph_prop_once(
        node_states,
        from_idx,
        to_idx,
        self._message_net,
        aggregation_module=self._aggregation_module,
        edge_features=edge_features)

    # optionally compute message vectors in the reverse direction
    if self._use_reverse_direction:
      if self._reverse_dir_param_different:
        self._reverse_message_net = snt.nets.MLP(
            self._edge_hidden_sizes,
            initializers={
                'w': tf.variance_scaling_initializer(
                    scale=self._edge_net_init_scale),
                'b': tf.zeros_initializer()},
            name='reverse_message_mlp')
      else:
        self._reverse_message_net = self._message_net

      reverse_aggregated_messages = graph_prop_once(
          node_states,
          to_idx,
          from_idx,
          self._reverse_message_net,
          aggregation_module=self._aggregation_module,
          edge_features=edge_features)

      aggregated_messages += reverse_aggregated_messages

    if self._layer_norm:
      aggregated_messages = snt.LayerNorm()(aggregated_messages)

    return aggregated_messages, new_edge_features


  def _compute_node_update(self,
                           node_states,
                           node_state_inputs,
                           node_features=None):
    """Computes node updates.

    Args:
      node_states: [n_nodes, node_state_dim] float tensor, input node states.
      node_state_inputs: list of tensors used to compute node updates.  Each
        element tensor should have shape [n_nodes, feat_dim], where feat_dim can
        be different from node_states. These tensors will be concatenated along
        the feature dimension.
      node_features: extra node features if provided, should be of size
        [n_nodes, extra_node_feat_dim] float tensor.

    Returns:
      updated_node_states: [n_nodes, node_state_dim] float tensor, the new node
        state tensor.

    Raises:
      ValueError: if node update type is not supported.
    """
    if self._node_update_type in ('mlp', 'residual'):
      node_state_inputs.append(node_states)
      
    if node_features is not None:
      node_state_inputs.append(node_features)

    if len(node_state_inputs) == 1:
      node_state_inputs = node_state_inputs[0]
    else:
      node_state_inputs = tf.concat(node_state_inputs, axis=-1)

    if self._node_update_type == 'gru':
      _, new_node_states = snt.GRU(self._node_state_dim)(
          node_state_inputs, node_states)
      return new_node_states
    else:
      mlp_output = snt.nets.MLP(
          self._node_hidden_sizes, name='node_mlp')(node_state_inputs)
      if self._layer_norm:
        mlp_output = snt.LayerNorm()(mlp_output)
      if self._node_update_type == 'mlp':
        return mlp_output
      elif self._node_update_type == 'residual':
        return node_states + mlp_output
      else:
        raise ValueError('node update type {} not supported'.format(self._node_update_type))

  def _build(self,
             node_states,
             from_idx,
             to_idx,
             edge_features=None,
             node_features=None):
    """Runs one round of propagation (message passing).

    Args:
      node_states: [n_nodes, input_node_state_dim] float tensor, node states.
      from_idx: [n_edges] int tensor, from node indices for each edge.
      to_idx: [n_edges] int tensor, to node indices for each edge.
      edge_features: if not None, should be [n_edges, edge_embedding_dim]
        tensor, edge features.
      node_features: extra node features if provided, should be of size
        [n_nodes, extra_node_feat_dim] float tensor, can be used to implement
        different types of skip connections.

    Returns:
      node_states: [n_nodes, node_state_dim] float tensor, new node states.
    """
    aggregated_messages, updated_edge_features = self._compute_aggregated_messages(
        node_states, from_idx, to_idx, edge_features=edge_features)
    
    # node_states: initial node feature for current round of message passing
    # aggregated_messages: messages from the message passing proporgation
    return self._compute_node_update(node_states,
                                     [aggregated_messages],
                                     node_features=node_features), updated_edge_features

## Graph output module


In [6]:
class GraphOutput(snt.AbstractModule):
  """Computes node representations and make node level predictions."""

  def __init__(self,
               node_hidden_sizes,
               output_dim=1,
               gated=True,
               aggregation_type='mean',
               name='graph_aggregator'):
    """Constructor.

    Args:
      node_hidden_sizes: the hidden layer sizes of the node transformation nets.
        The last element is the size of the aggregated graph representation.
      output_dim: int. Output dimension for each node.
      gated: set to True to do gated aggregation, False not to.
      aggregation_type: one of {sum, max, mean, sqrt_n}.
      name: name of this module.
    """
    super(GraphOutput, self).__init__(name=name)

    self._node_hidden_sizes = node_hidden_sizes
    self._graph_state_dim = node_hidden_sizes[-1]
    self._gated = gated
    self._output_dim = output_dim

  def _build(self, node_states, graph_idx, n_graphs):
    """Compute node representation and level predictions.

    Args:
      node_states: [n_nodes, node_state_dim] float tensor, node states of a
        batch of graphs concatenated together along the first dimension.
      graph_idx: [n_nodes] int tensor, graph id for each node.
      n_graphs: integer, number of graphs in this batch.

    Returns:
      node_states: [n_nodes, node_state_dim] float tensor, the node level 
        prediction.
    """
    node_hidden_sizes = self._node_hidden_sizes
    if self._gated:
      node_hidden_sizes[-1] = self._graph_state_dim * 2

    node_states_g = snt.nets.MLP(
        node_hidden_sizes, name='node_state_g_mlp')(node_states)

    if self._gated:
      gates = tf.nn.sigmoid(node_states_g[:, :self._graph_state_dim])
      node_states_g = node_states_g[:, self._graph_state_dim:] * gates 

    # Transforms the node states for final node level output
    node_states_output = snt.nets.MLP(
        [self._output_dim], name='node_state_transform_mlp')(node_states_g) 
    
    return node_states_output    

## Graph model

Now we can put the three modules all together with encoder, message-passing, graph aggregation modules.

In [7]:
AGGREGATION_TYPE = {
    'sum': tf.math.unsorted_segment_sum,
    'mean': tf.math.unsorted_segment_mean,
    'sqrt_n': tf.math.unsorted_segment_sqrt_n,
    'max': tf.math.unsorted_segment_max,
}

class GraphSTNet(snt.AbstractModule):
  """A ST-GNN to learn both spatial and temporal interactions in the network."""

  def __init__(self,
               encoder,
               output_module,
               node_state_dim,
               edge_hidden_sizes,
               node_hidden_sizes,
               n_prop_layers,
               label_seq_len=7,
               label_dim=1,
               share_prop_params=False,
               edge_net_init_scale=0.1,
               node_update_type='residual',
               use_reverse_direction=True,
               reverse_dir_param_different=True,
               layer_norm=False,
               aggregation_type='mean',
               name='graph_st_net'):
    """Constructor.

    Args:
      encoder: GraphEncoder, encoder that maps features to embeddings.
      aggregator: GraphAggregator, aggregator that produces node level 
        predictions.
      node_state_dim: dimensions of node states.
      edge_hidden_sizes: sizes of the hidden layers of the edge message nets.
      node_hidden_sizes: sizes of the hidden layers of the node update nets.
      n_prop_layers: number of graph propagation layers.
      share_prop_params: set to True to share propagation parameters across all
        graph propagation layers, False not to.
      edge_net_init_scale: scale of initialization for the edge message nets.
      node_update_type: type of node updates, one of {mlp, gru, residual}.
      use_reverse_direction: set to True to also propagate messages in the
        reverse direction.
      reverse_dir_param_different: set to True to have the messages computed
        using a different set of parameters than for the forward direction.
      layer_norm: set to True to use layer normalization in a few places.
      name: name of this module.
    """
    super(GraphSTNet, self).__init__(name=name)

    self._encoder = encoder
    self._output = output_module
    self._node_state_dim = node_state_dim
    self._edge_hidden_sizes = edge_hidden_sizes
    self._node_hidden_sizes = node_hidden_sizes
    self._n_prop_layers = n_prop_layers
    self._share_prop_params = share_prop_params
    self._edge_net_init_scale = edge_net_init_scale
    self._node_update_type = node_update_type
    self._use_reverse_direction = use_reverse_direction
    self._reverse_dir_param_different = reverse_dir_param_different
    self._layer_norm = layer_norm
    self._aggregation_module = AGGREGATION_TYPE[aggregation_type]

    self._prop_layers = []
    self._layer_class = GraphPropLayer

    self._label_seq_len = label_seq_len
    self._label_dim = label_dim


  def _build_layer(self, layer_id):
    """Build one layer in the network."""
    return self._layer_class(
        self._node_state_dim,
        self._edge_hidden_sizes,
        self._node_hidden_sizes,
        edge_net_init_scale=self._edge_net_init_scale,
        node_update_type=self._node_update_type,
        use_reverse_direction=self._use_reverse_direction,
        reverse_dir_param_different=self._reverse_dir_param_different,
        layer_norm=self._layer_norm,
        aggregation_module=self._aggregation_module,
        name='graph_prop_%d' % layer_id)

  def _apply_layer(self,
                   layer,
                   node_states,
                   from_idx,
                   to_idx,
                   graph_idx,
                   n_graphs,
                   edge_features):
    """Apply one layer on the given inputs."""
    del graph_idx, n_graphs
    return layer(node_states, from_idx, to_idx, edge_features=edge_features)


  def _build(self,
             node_features,
             edge_features,
             from_idx,
             to_idx,
             graph_idx,
             n_graphs):
    """Compute node prediction.

    Args:
      node_features: [n_nodes, node_feat_dim] float tensor.
      edge_features: [n_edges, edge_feat_dim] float tensor.
      from_idx: [n_edges] int tensor, index of the from node for each edge.
      to_idx: [n_edges] int tensor, index of the to node for each edge.
      graph_idx: [n_nodes] int tensor, graph id for each node.
      n_graphs: int, number of graphs in the batch.

    Returns:
      node_prediction: [n_nodes, label_dim] float tensor.
    """
    if len(self._prop_layers) < self._n_prop_layers:
      # build the layers
      for i in range(self._n_prop_layers):
        if i == 0 or not self._share_prop_params:
          layer = self._build_layer(i)
        else:
          layer = self._prop_layers[0]
        self._prop_layers.append(layer)

    node_states, edge_states = self._encoder(node_features, edge_features)

    layer_outputs = [node_states]
    curre_edge_states = edge_states
    for layer in self._prop_layers:
      node_states, new_edge_states = self._apply_layer(
          layer,
          node_states,
          from_idx,
          to_idx,
          graph_idx,
          n_graphs,
          curre_edge_states)
      curre_edge_states = new_edge_states
      layer_outputs.append(node_states)

    # these tensors may be used e.g. for visualization
    self._layer_outputs = layer_outputs

    # predictions at each node level, node*(Pred_len)
    node_pred = self._output(node_states, graph_idx, n_graphs)  
    return node_pred

  def reset_n_prop_layers(self, n_prop_layers):
    """Sets n_prop_layers to the provided new value.

    This allows us to train with certain number of propagation layers and
    evaluate with a different number of propagation layers.

    This only works if n_prop_layers is smaller than the number used for
    training, or when share_prop_params is set to True, in which case this can
    be arbitrarily large.

    Args:
      n_prop_layers: the new number of propagation layers to set.
    """
    self._n_prop_layers = n_prop_layers

  @property
  def n_prop_layers(self):
    return self._n_prop_layers

  def get_layer_outputs(self):
    """Get the outputs at each layer."""
    if hasattr(self, '_layer_outputs'):
      return self._layer_outputs
    else:
      raise ValueError('No layer outputs available.')    

In [8]:
mymodel = GraphSTNet(GraphEncoder([8], [8]),
                     GraphOutput([8]),
                     8, [8], [8], 2 )
mymodel

## Define loss function

In [9]:
def compute_loss(y, yhat, loss_type='mse'):
  # yhat: node_num*seq_len*label_dim
  if loss_type == 'mse':
    d_case_loss = tf.keras.losses.MSE(y, yhat[:,:,0])   
    return tf.reduce_mean(d_case_loss)       
  else:
    raise ValueError('Unknown loss_type %s' % loss_type)


#### Test the loss function

In [10]:
# Test loss function
n_nodes = 5
y = np.random.normal(size=[n_nodes,  7])
yhat = np.random.normal(size=[n_nodes, 7, 3])

total_loss = compute_loss(y, yhat, loss_type='mse')
sess = tf.Session()
sess.run(total_loss)

1.5563361457923817

# Graph Datasets
To represent a graph, we need node level and edge level information, from_idx, and to_idx to describe the connectivity in the graph. Because training and evaluation is usually done in batches, data from multiple graphs are concatinated together, therefore, we also need describe the total number of graphs and the index of graph that each node belongs to.

The following data structure captures what we have discussed.

In [11]:
list_data_info = ['from_idx',
                  'to_idx',
                  'node_features',
                  'edge_features',
                  'graph_idx',
                  'n_graphs',
                  'node_labels']
GraphData = collections.namedtuple('GraphData', list_data_info)

## Simulated dataset 
We would like our graph data to capture both spatial and temporal information. The temporal information is wrapped in node feature and edge features; the spatial interaction is encoded naturally in the graph structure. 

In [12]:
class GraphDataset():
  """Graph dataset for node level predictions."""

  def __init__(self,
               n_nodes_range,
               n_edge_range):
    """Constructor.

    Args:
      n_nodes_range: a tuple (n_min, n_max).  The minimum and maximum number of
        nodes in a graph to generate.
      p_edge_range: a tuple (p_min, p_max).  The minimum and maximum edge
        probability.
      n_changes_positive: the number of edge substitutions for a pair to be
        considered positive (similar).
      n_changes_negative: the number of edge substitutions for a pair to be
        considered negative (not similar).
      permute: if True (default), permute node orderings in addition to
        changing edges; if False, the node orderings across a pair or triplet of
        graphs will be the same, useful for visualization.
    """
    self._n_min, self._n_max = n_nodes_range
    self._e_min, self._e_max = n_edge_range

  def _get_graph(self):
    """Generates one graph."""
    seq_len, node_feat_dim, edge_feat_dim = 7, 3, 2
    n_nodes = np.random.randint(self._n_min, self._n_max + 1)
    n_edges = np.random.randint(self._e_min, self._e_max+1)
    nodes = np.random.normal(size=[n_nodes,  seq_len, node_feat_dim])
    edges = np.random.normal(size=[n_edges, seq_len, edge_feat_dim])
    senders = np.random.randint(0, n_nodes, size=[n_edges]) 
    receivers = np.random.randint(0, n_nodes, size=[n_edges]) 

    # Assume the label dim is 1, which is usually the case.
    node_labels = np.random.normal(size=[n_nodes, 1])

    return GraphData(from_idx=senders,
                     to_idx=receivers,
                     node_features=nodes,
                     edge_features=edges,
                     node_labels=node_labels,
                     graph_idx=np.zeros(n_nodes, dtype=np.int32),
                     n_graphs=1)


  def _get_one_batch(self, batch_size):
    """Build a batch of GraphData objects into one GraphData.
    
    This is done by concatinating the different data in each GraphData in the 
    first dimension. Graph index needs to be updated dynamically to distinguish
    between different graphs.
    """
    from_idx = []
    to_idx = []
    graph_idx = []
    node_feat = []
    edge_feat = []
    node_label =[]

    n_total_nodes = 0
    n_total_edges = 0
    for i in range(batch_size):
      g = self._get_graph()
      n_nodes = g.node_features.shape[0]
      n_edges = g.edge_features.shape[0]

      # Update the node indices for the edges
      from_idx.append(g.from_idx + n_total_nodes)
      to_idx.append(g.to_idx + n_total_nodes)
      graph_idx.append(np.ones(n_nodes, dtype=np.int32) * i)
      node_feat.append(g.node_features)
      edge_feat.append(g.edge_features)
      node_label.append(g.node_labels)

      n_total_nodes += n_nodes
      n_total_edges += n_edges

    return GraphData(
        from_idx=np.concatenate(from_idx, axis=0),
        to_idx=np.concatenate(to_idx, axis=0),
        node_features=np.concatenate(node_feat, axis=0),
        edge_features=np.concatenate(edge_feat, axis=0),
        graph_idx=np.concatenate(graph_idx, axis=0),
        node_labels=np.concatenate(node_label, axis=0),
        n_graphs=batch_size)

In [13]:
# Test the dataset generation.
graph_gen = GraphDataset((3, 3), (2, 2))
graph_gen._get_one_batch(3) # this looks good

GraphData(from_idx=array([2, 0, 5, 3, 7, 7]), to_idx=array([2, 0, 5, 3, 6, 8]), node_features=array([[[ 6.49750554e-01, -5.28160264e-01, -2.73474921e-01],
        [ 2.27882079e+00, -7.55231331e-01, -1.01548164e+00],
        [-2.29521214e-01, -4.74420000e-01, -5.42315706e-01],
        [-1.64376636e+00, -4.75936718e-01, -2.45602534e-01],
        [ 1.26626501e+00,  3.32855644e-02,  6.18323443e-02],
        [ 8.73964463e-01, -1.05059542e+00,  4.99566536e-02],
        [-5.01248912e-01, -1.17138958e+00, -1.71597137e+00]],

       [[-4.79838393e-02, -6.99212662e-02, -1.55647892e+00],
        [ 4.93130044e-01,  7.92857523e-01, -2.61839908e-01],
        [-1.46919304e+00,  1.07571456e+00, -3.57115584e-01],
        [ 5.34141818e-01, -7.24781613e-01, -1.02628521e+00],
        [ 2.08674819e+00, -1.84461728e+00,  3.29698717e-02],
        [-1.31372144e+00,  2.99319106e-01,  1.20925408e+00],
        [ 2.07811860e+00, -3.21434481e+00, -1.22254428e+00]],

       [[ 8.86901997e-02, -1.67209707e-01,  1.19

## Random spatio-temporal dataset
Here we show the steps to generate the spatio-temporal dataset for ST-GNN training. The assumptions are:
* we have a dataset with temporal observations on each nodes at a certain frequency, e.g. daily, weekly, etc.
* we have a dataset with temporal observations on each connectivity(directed edge bewteen nodes) at the same frequency with node level observations.

This part is just an illustration of how to generate GraphDataset from a spatio-temporal input, the data itself does not carry any meanings or spatio-temporal interations. With this example, it is easy to replace the above random input with realistic spatio-temporal dataset with the same format to run training. 

### Create node and index mapping

In [14]:
node_names_idxs = [(chr(i), i-97) for i in range(97, 123)]

node_idx2name, node_name2idx = {}, {}
for name, idx in node_names_idxs:
  node_name2idx[name] = idx
  node_idx2name[idx] = name

num_nodes = len(node_names_idxs)
node_names = node_name2idx.keys()
print(f'Total {num_nodes} nodes.')
print(f'The nodes are {node_names}')

Total 26 nodes.
The nodes are dict_keys(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])


### Create node level data

In [15]:
num_days = 200
year = 2020
days = [dayi for dayi in range(num_days)]
dates = [(datetime.datetime(year, 1, 1) + datetime.timedelta(dayi)).date().strftime('%Y-%m-%d') for dayi in range(num_days)]

# Create node level daily observations
# date  day node_name	feat1	feat2
nodes_history = []
num_node_feats = 2
node_data = None
for i in range(num_nodes):
  nodes_data_array = np.random.normal(size=[num_days, num_node_feats])
  curr_df = pd.DataFrame({'date': dates,
                          'day': days,
                          'node_idx': i,
                          'node_name': node_idx2name[i],
                          'node_feat1': nodes_data_array[:, 0],
                          'node_feat2': nodes_data_array[:, 1]})
  if node_data is None:
    node_data = curr_df.copy()
  else:
    node_data = pd.concat([node_data, curr_df], axis=0)

print('A sample of node data: ')
node_data.head(5)

A sample of node data: 


,date,day,node_idx,node_name,node_feat1,node_feat2
0,2020-01-01,0,0,a,-1.177029,0.398712
1,2020-01-02,1,0,a,-3.063158,0.405811
2,2020-01-03,2,0,a,2.082045,0.675790
3,2020-01-04,3,0,a,-0.806469,-0.953733
4,2020-01-05,4,0,a,-0.593832,0.856214


### Create edge level data

In [16]:
# Create edge level daily observations
# date day src dest	edge_feat1 edge_feat2

# define the range of edge numbers in a graph
min_edges, max_edges = 10, 40
num_edge_feats = 2

# non self-loop candidates
node_ids = node_idx2name.keys()
pairs = list(itertools.combinations(node_ids, 2))
random.shuffle(pairs)
total_pairs = len(pairs)


edge_data = None
for i in range(num_days):
  # num of non self-loops edges
  n_edges = np.random.randint(min_edges, max_edges+1)
  # get n_edges random unique pairs
  edge_idxs = random.sample(pairs, n_edges)
  edge_src = [e_idx[0] for e_idx in edge_idxs]
  edge_dest = [e_idx[1] for e_idx in edge_idxs]
  edge_src_name = [node_idx2name[node_i] for node_i in edge_src]
  edge_dest_name = [node_idx2name[node_i] for node_i in edge_dest]

  edge_data_array = np.random.normal(size=[n_edges, num_edge_feats])
  curr_df = pd.DataFrame({'date': (datetime.datetime(year, 1, 1) + datetime.timedelta(i)).date().strftime('%Y-%m-%d'),
                          'day': i,
                          'src': edge_src,
                          'dest': edge_dest,
                          'src_name': edge_src_name,
                          'dest_name': edge_dest_name,                          
                          'edge_feat1': edge_data_array[:, 0],
                          'edge_feat2': edge_data_array[:, 1]})
  if edge_data is None:
    edge_data = curr_df.copy()
  else:
    edge_data = pd.concat([edge_data, curr_df], axis=0)

print('A sample of edge data:')
edge_data.head(5)

A sample of edge data:


,date,day,src,dest,src_name,dest_name,edge_feat1,edge_feat2
0,2020-01-01,0,14,21,o,v,0.167776,2.405669
1,2020-01-01,0,6,15,g,p,0.317989,1.780007
2,2020-01-01,0,3,17,d,r,0.557793,-1.235262
3,2020-01-01,0,16,25,q,z,-0.180008,-0.612154
4,2020-01-01,0,7,14,h,o,-0.910204,0.202476


## Create GraphStDataset from the random data

### Implement the GraphSTDataset class

In [17]:
class GraphStDataset():
  """Graph spatio-temporal dataset for node level predictions."""

  def __init__(self,
               node_history_len,
               node_target_len,
               edge_history_len,
               node_idx2name, 
               node_name2idx,
               node_data,
               edge_data,
               day_threshold):
    """Constructor.

    Args:
      node_history_len: int. The length of past node observations to include.
      node_target_len: int. The prediction is node_target_len away.
      edge_history_len: int. The length of past edge observations to include.
      node_idx2name: a dict. A dict that maps node index to its name.
      node_name2idx: a dict. A dict that maps node name to its index.
      node_data: a pandas DataFrame. With columns as day, date, node_name, 
        node_feat1, node_feat2.
      day_threshold: int. The cutoff day to use to compute statistics, so we 
        can normalize the column values.

    """
    self._node_history_len = node_history_len
    self._edge_history_len = edge_history_len
    self._node_target_len = node_target_len
    self._node_idx2name = node_idx2name
    self._node_name2idx = node_name2idx
    self._node_data = node_data
    self._edge_data = edge_data
    self._node_data_dict = self._build_node_data_dict(node_data, day_threshold)

  def _scale_column_val(self, df, col_name, day_threshold):
    min_val = df.loc[df['day']<=day_threshold][col_name].min()
    max_val = df.loc[df['day']<=day_threshold][col_name].max()
    df['scaled_'+col_name] = (df[col_name]-min_val)/(max_val-min_val)  
    return df, min_val, max_val

  def _build_node_data_dict(self, node_data, day_threshold):
    """Processes and builds a dict that maps node_name to its temporal 
      observations. Also normalize from avaiable observations before threshold.
      We can add other derived features, e.g. delta of a certain feature.

    Args:
      node_data: a pandas DataFrame. With columns as day, date, node_name, 
        node_feat1, node_feat2.
      day_threshold: int. The cutoff day to use to compute statistics, so we 
        can normalize the column values.

    Returns:
      node_data_dict: a dict. That maps node_name to its temporal observations.
      node_data_scaling_metrics: a dict. That stores the scaling metrics for 
        each node.
    """
    node_data_dict = {}
    self._node_data_scaling_metrics = {}
    for node in node_data.node_name.unique():
      curr_node_data = node_data.loc[node_data['node_name']==node].copy()
      curr_node_data.sort_values(by=['day'], inplace=True)

      curr_node_data['node_feat1_delta'] = curr_node_data.node_feat1.diff()
      curr_node_data['node_feat2_delta'] = curr_node_data.node_feat1.diff()

      curr_node_data, min_val, max_val = self._scale_column_val(curr_node_data, 
                                                          'node_feat1_delta', 
                                                          day_threshold)
      self._node_data_scaling_metrics[node+'node_feat1_delta']=(min_val, max_val)

      curr_node_data, min_val, max_val = self._scale_column_val(curr_node_data, 
                                                          'node_feat2_delta', 
                                                          day_threshold)
      self._node_data_scaling_metrics[node+'node_feat2_delta']=(min_val, max_val)

      # Removes the first row, since it has NaN for deltas
      node_data_dict[node] = curr_node_data[1:].copy()
    return node_data_dict

  def _get_graph(self, 
                 day_number,
                 node_feat_names, 
                 edge_feat_names, 
                 node_label_names):
    """Generates one graph.

    Args:
      day_number: an int. On which day the graph is built upon. The resulting
        graph should have historical observations <= day_number, and lable
        should be future observations > day_number.
      node_feat_names: a list of str. Node features to be included. 
      edge_feat_names: a list of str. Edgge features to be included. 
      node_label_names: a list of str. Node columns to be included as labels. 
    """

    # seq_len, node_feat_dim, edge_feat_dim = 7, 3, 2
    node_data_dict = self._node_data_dict
    node_names = list(node_data_dict.keys())
    n_nodes = len(node_names)

    node_features = np.zeros((n_nodes, self._node_history_len, len(node_feat_names)))
    node_targets = np.zeros((n_nodes, 1, len(node_label_names)))

    # Processes node    
    for i in range(n_nodes):
      curr_data = node_data_dict[node_idx2name[i]].copy()
      hist_data = curr_data.loc[(curr_data['day']<=day_number) & 
                                (curr_data['day']>day_number-self._node_history_len)].copy().sort_values(by=['day'], inplace=False)
      time_df = pd.DataFrame({'day': 
                              list(range(day_number-self._node_history_len+1, day_number+1))})
      hist_data = pd.merge(time_df, hist_data, left_on='day', right_on='day', how='left')
      hist_data.ffill(inplace=True)

      # hist_data
      hist_data.fillna(0, inplace=True)
      if hist_data.shape[0]>0:
        node_features[i, :, :] = hist_data[node_feat_names].values
      else: 
        node_features[i, :, :] = np.zeros(shape=[self._node_history_len, 
                                          len(node_feat_names)])
        
      # get delta between curr+target_len and curr
      forecast_data = curr_data.loc[(curr_data['day']>=day_number) & 
                                (curr_data['day']<=day_number+self._node_target_len)].copy().sort_values(by=['day'], inplace=False)
      time_df = pd.DataFrame({'day': 
                              list(range(day_number, day_number+self._node_target_len+1))})
      forecast_data = pd.merge(time_df, forecast_data, left_on='day', right_on='day', how='left')
      forecast_data.ffill(inplace=True)

      # forecast_data
      forecast_data.fillna(0, inplace=True)
      if forecast_data.shape[0]>0:
        # just get the quantity on (t+n)
        curr_df = forecast_data[node_label_names].copy()
        cu_val = curr_df.loc[curr_df.shape[0]-1]
        node_targets[i, :, :] = np.array([cu_val.values]) 
      else: 
        # uses zeros as unknowns
        node_targets[i, :, :] = np.zeros(shape=[1, len(node_label_names)])
        
    nodes = np.asarray(node_features)
    node_labels = np.asarray(node_targets)

    # Processes edge
    # each day_number, find all unique pairs in the past history
    # and construct edge features.
    edge_data_for_day_range = edge_data.loc[(edge_data['day']<=day_number) &
                                            (edge_data['day']>day_number-self._edge_history_len)].copy()
    unique_edge_pairs = edge_data_for_day_range[['src', 'dest']].copy().drop_duplicates()

    edge_features = np.zeros((unique_edge_pairs.shape[0], 
                              self._edge_history_len, 
                              len(edge_feat_names)))
    from_idx = []
    to_idx = []
    for idx, row in unique_edge_pairs.iterrows():
      src, dest = row.src, row.dest
      # add receiver and sender
      from_idx.append(src)
      to_idx.append(dest)

      # add edge features
      curr_edge_data = edge_data.loc[(edge_data['day']>day_number-self._edge_history_len) & 
                                (edge_data['day']<=day_number)&
                                (edge_data['src']==src)&
                                (edge_data['dest']==dest)].copy().sort_values(by=['day'], inplace=False)
      time_df = pd.DataFrame({'day': 
                              list(range(day_number-self._edge_history_len+1, day_number+1))})   
      curr_edge_data = pd.merge(time_df, curr_edge_data, left_on='day', right_on='day', how='left')
      curr_edge_data.ffill(inplace=True)

      # hist_edge_data
      curr_edge_data.fillna(0, inplace=True)
      if curr_edge_data.shape[0]>0:
        edge_features[i, :, :] = curr_edge_data[edge_feat_names].values
      else: 
        edge_features[i, :, :] = np.zeros(shape=[self._edge_history_len, len(edge_feat_names)])

    edges = np.asarray(edge_features)
    senders = np.asarray(from_idx) 
    receivers = np.asarray(to_idx)  

    return GraphData(from_idx=senders,
                     to_idx=receivers,
                     node_features=nodes,
                     edge_features=edges,
                     node_labels=node_labels,
                     graph_idx=np.ones(n_nodes, dtype=np.int32)*day_number,
                     n_graphs=1)
    
  def build_batches(self, node_feat_names, edge_feat_names, node_label_names,
                    min_day, max_day, batch_size, ):
    """Generates batches of graphs.

    Args:
      min_day, max_day: integers. The date range to build graphs, one graph for
        each day.
    """
    days = range(min_day, max_day+1, batch_size)
    batched_data = [] # len=total_num_batches
    for d in days:
      one_batch = []
      for curr_d in range(d, d+batch_size):
        if curr_d<=max_day:
          one_batch.append(self._get_graph(curr_d, 
                                           node_feat_names, 
                                           edge_feat_names, 
                                           node_label_names))

      batched_data.append(self._pack_batch(one_batch))
    return batched_data

  def _pack_batch(self, one_batch):
    from_idx = []
    to_idx = []
    graph_idx = []
    node_feat = []
    edge_feat = []
    node_label =[]

    n_total_nodes = 0
    n_total_edges = 0
    for g in one_batch:
      n_nodes = g.node_features.shape[0]
      n_edges = g.edge_features.shape[0]
      # shift the node indices for the edges
      from_idx.append(g.from_idx + n_total_nodes)
      to_idx.append(g.to_idx + n_total_nodes)
      graph_idx.append(g.graph_idx)
      node_feat.append(g.node_features)
      edge_feat.append(g.edge_features)
      node_label.append(g.node_labels)

      n_total_nodes += n_nodes
      n_total_edges += n_edges

    return GraphData(
        from_idx=np.concatenate(from_idx, axis=0),
        to_idx=np.concatenate(to_idx, axis=0),
        # this task only cares about the structures, the graphs have no features
        node_features=np.concatenate(node_feat, axis=0),
        edge_features=np.concatenate(edge_feat, axis=0),
        graph_idx=np.concatenate(graph_idx, axis=0),
        node_labels=np.concatenate(node_label, axis=0),
        n_graphs=len(one_batch))

### Test the data generation

In [18]:
# test generating one graph
graph_st_gen = GraphStDataset(node_history_len=7, 
                              edge_history_len=7,  
                              node_target_len=5, 
                              day_threshold=50,
                              node_idx2name=node_idx2name, 
                              node_name2idx=node_name2idx,
                              node_data=node_data, 
                              edge_data=edge_data)

node_feat_names=['node_feat1_delta', 'node_feat2_delta']
edge_feat_names=['edge_feat1', 'edge_feat2']
lable_names=['node_feat1_delta']
min_day, max_day=50, 60
batch_size=5
test_batch = graph_st_gen.build_batches(node_feat_names,  
                                        edge_feat_names, 
                                        lable_names,
                                        min_day, max_day, batch_size)

assert test_batch[1].node_features.shape[2]==len(node_feat_names)
test_batch[1].node_features.shape
test_batch[1].node_labels.shape

(130, 7, 2)

(130, 1, 1)

# Build the model

To build the model, we need:

* Set up the placeholders, default model configuration.
* Build the model, build the computation graphs for training and eval, build the metrics and statistics to monitor.
* The graphs are batched as a sequence of graphs.

### Placeholders

In [19]:
def build_placeholders(node_feature_dim, edge_feature_dim, label_dim,
                       node_seq_len, edge_seq_len, label_seq_len):
  """Builds the placeholders for the model.

  Args:
    node_feature_dim: int.
    edge_feature_dim: int.

  Returns:
    placeholders: a placeholder name -> placeholder tensor dict.
  """
  return {
      'node_features': tf.placeholder(tf.float32, [None, node_seq_len, node_feature_dim]),
      'edge_features': tf.placeholder(tf.float32, [None, edge_seq_len, edge_feature_dim]),
      'from_idx': tf.placeholder(tf.int32, [None]),
      'to_idx': tf.placeholder(tf.int32, [None]),
      'graph_idx': tf.placeholder(tf.int32, [None]),
      'labels': tf.placeholder(tf.float32, [None, 1, label_dim])
      }


### Config

In [20]:
def get_default_config():
  """The default configs. Make updates accordingly for the learning task."""

  ###!!! Note to update this configuration!!!#########
  day_threshold = 50
  # Past history info to use
  node_seq_len = 14
  edge_seq_len = 14
  # Predict for horizon steps away
  horizon = 7 

  # Control the day range and batch size to use as train and test datasets
  # An example is like following:
  min_day_train=60  + node_seq_len #60
  max_day_train=126 - horizon -1
  batch_size_train=32

  min_day_test=126 - horizon
  max_day_test=150 - horizon  
  batch_size_test=5  
  ###################################################
  label_seq_len=horizon
  label_dim=1

  node_state_dim = 16
  edge_state_dim = 16
  graph_rep_dim = 16

  graph_st_config = dict(
      node_state_dim=node_state_dim,
      edge_hidden_sizes=[node_state_dim*2, node_state_dim],
      node_hidden_sizes=[node_state_dim],
      label_seq_len = label_seq_len,
      label_dim=label_dim,
      n_prop_layers=6,
      # set to False to not share parameters across message passing layers
      share_prop_params=True,
      # initialize message MLP with small parameter weights to prevent
      # aggregated message vectors blowing up, alternatively we could also use
      # e.g. layer normalization to keep the scale of these under control.
      edge_net_init_scale=0.1,
      # other types of update like `mlp` and `residual` can also be used here.
      node_update_type='gru',
      # set to False if your graph already contains edges in both directions.
      use_reverse_direction=False,
      reverse_dir_param_different=False,
      aggregation_type='mean',
      layer_norm=True)

  return dict(
      data_param=dict(node_seq_len=node_seq_len, 
                      edge_seq_len=edge_seq_len,
                      label_seq_len=label_seq_len,
                      label_dim=label_dim,
                      day_threshold=day_threshold,
                      horizon=horizon),
      encoder=dict(
          node_hidden_sizes=[node_state_dim],
          edge_hidden_sizes=[edge_state_dim]),
      output=dict(
          node_hidden_sizes=[graph_rep_dim],
          gated=True,
          output_dim=1),
      graph_st_net=graph_st_config,
      training=dict(
          min_day=min_day_train,
          max_day=max_day_train,
          batch_size=batch_size_train,
          learning_rate=1e-2,
          loss='mse',
          graph_vec_regularizer_weight=1e-6,
          clip_value=10.0,
          n_training_steps=10000,
          print_after=1,
          eval_after=1),
      evaluation=dict(
          min_day=min_day_test,
          max_day=max_day_test,
          batch_size=batch_size_test),
      seed=8,
      )

### Model

In [21]:
def build_model(config, node_feature_dim, edge_feature_dim, label_dim):
  """Create model for training and evaluation.

  Args:
    config: a dict of configs, like the one created by the `get_default_config`.
    node_feature_dim: int, dimension of node features.
    edge_feature_dim: int, dimension of edge features.
    label_dim: int, dimension of node level labels.

  Returns:
    tensors: a (potentially nested) name => tensor dict.
    placeholders: a (potentially nested) name => tensor dict.
    model: a GraphSTNet instance.

  Raises:
    ValueError: if the specified model or training settings are not supported.
  """
  encoder = GraphEncoder(**config['encoder'])
  print('Done with encoder.')
  output_module = GraphOutput(**config['output'])

  model = GraphSTNet(
      encoder, 
      output_module, 
      **config['graph_st_net'])
  print('Done with GraphSTNet.')

  training_n_graphs_in_batch = config['training']['batch_size']

  node_seq_len = config['data_param']['node_seq_len']
  edge_seq_len = config['data_param']['node_seq_len']
  label_seq_len = config['data_param']['label_seq_len']
  placeholders = build_placeholders(node_feature_dim, 
                                    edge_feature_dim, 
                                    label_dim,
                                    node_seq_len,
                                    edge_seq_len,
                                    1)

  # training
  model_inputs = placeholders.copy()
  del model_inputs['labels']
  model_inputs['n_graphs'] = training_n_graphs_in_batch

  print('Model_inputs: {}'.format(model_inputs))

  # run model on the input, should have nodel level prediction
  train_pred = model(**model_inputs)  
  # print('graph_vector output: {}\n'.format(graph_vectors))
  loss = compute_loss(train_pred, placeholders['labels'],
                      loss_type=config['training']['loss'])

  optimizer = tf.train.AdamOptimizer(
      learning_rate=config['training']['learning_rate'])
  grads_and_params = optimizer.compute_gradients(loss)
  grads, params = zip(*grads_and_params)
  grads, _ = tf.clip_by_global_norm(grads, config['training']['clip_value'])
  train_step = optimizer.apply_gradients(zip(grads, params))

  grad_scale = tf.global_norm(grads)
  param_scale = tf.global_norm(params)

  print('Done with optimizer and model.')
  # evaluation
  model_inputs['n_graphs'] = config['evaluation']['batch_size'] 
  eval_pred = model(**model_inputs)
  mse_loss = compute_loss(eval_pred,
                          placeholders['labels'],
                          loss_type=config['training']['loss'])

  return {
      'train_step': train_step,
      'metrics': {
          'training': {
              'loss': loss,
              'grad_scale': grad_scale,
              'param_scale': param_scale,
          },
          'validation': {
              'mse_loss': mse_loss,
          },
      },
  }, placeholders, model

# Define utitlity functions for the training pipeline

In [22]:
def build_datasets(config, node_feat_names, edge_feat_names, node_label_name):
  """Builds the batched datasets for both training and evaluation."""
  config = copy.deepcopy(config)
  data_param = config['data_param']
  node_seq_len = data_param['node_seq_len']
  edge_seq_len = data_param['edge_seq_len']
  label_seq_len = data_param['label_seq_len']
  
  graph_st_gen = GraphStDataset(node_history_len=node_seq_len, 
                                edge_history_len=edge_seq_len,  
                                node_target_len=label_seq_len, 
                                day_threshold=data_param['day_threshold'],
                                node_idx2name=node_idx2name, 
                                node_name2idx=node_name2idx,
                                node_data=node_data, 
                                edge_data=edge_data)

  node_feat_names=['node_feat1_delta', 'node_feat2_delta']
  edge_feat_names=['edge_feat1', 'edge_feat2']
  lable_names=['node_feat1_delta']  
  
  min_day, max_day = config['training']['min_day'], config['training']['max_day']
  batch_size = config['training']['batch_size']
  training_set = graph_st_gen.build_batches(node_feat_names, 
                                            edge_feat_names, 
                                            node_label_name,
                                            min_day, max_day,
                                            batch_size
                                            )
  
  min_day, max_day = config['evaluation']['min_day'], config['evaluation']['max_day']
  batch_size = config['evaluation']['batch_size']  
  validation_set = graph_st_gen.build_batches(node_feat_names, 
                                              edge_feat_names, 
                                              node_label_name,
                                              min_day, max_day,
                                              batch_size)
  
  return training_set, validation_set, graph_st_gen

def fill_feed_dict(placeholders, batch):
  """Creates a feed dict for the given batch of data.

  Args:
    placeholders: a dict of placeholders.
    batch: a batch of data, should be a `GraphData` instance.

  Returns:
    feed_dict: a feed_dict that can be used in a session run call.
  """
  if isinstance(batch, GraphData):
    graphs = batch
    labels = None
  else:
    graphs, labels = batch  
  feed_dict = {
      placeholders['node_features']: graphs.node_features,
      placeholders['edge_features']: graphs.edge_features,
      placeholders['from_idx']: graphs.from_idx,
      placeholders['to_idx']: graphs.to_idx,
      placeholders['graph_idx']: graphs.graph_idx,
      placeholders['labels'] : graphs.node_labels,
  }  
  return feed_dict

def evaluate(sess, eval_metrics, placeholders, validation_set, batch_size):
  """Evaluates model performance on the given validation set.

  Args:
    sess: a `tf.Session` instance used to run the computation.
    eval_metrics: a dict containing tensor 'mse_loss'.
    placeholders: a placeholder dict.
    validation_set: a collection of `GraphDataset` instance, calling `pairs` and
      `triplets` functions with `batch_size` creates iterators over a finite
      sequence of batches to evaluate on.
    batch_size: number of batches to use for each session run call.

  Returns:
    metrics: a dict of metric name => value mapping.
  """
  accumulated_mse_loss = []
  # batch=validation_set._get_one_batch(batch_size)

  for batch in validation_set:
    feed_dict = fill_feed_dict(placeholders, batch)
    mse_loss = sess.run(eval_metrics['mse_loss'], feed_dict=feed_dict)
    accumulated_mse_loss.append(mse_loss)

  return {
      'mse_loss': np.mean(accumulated_mse_loss),
  }

# Run pipeline

In [23]:
config = get_default_config()

config['training']['n_training_steps'] = 5000

In [24]:
# Run this if you want to run the code again, otherwise tensorflow would
# complain that you already created the same graph and the same variables.
tf.reset_default_graph()

# Set random seeds
seed = config['seed']
random.seed(seed)
np.random.seed(seed + 1)
tf.set_random_seed(seed + 2)


node_feat_names=['node_feat1_delta', 'node_feat1_delta']
edge_feat_names=['edge_feat1_delta', 'edge_feat2_delta']
node_label_name=['node_feat1_delta']

training_batch_set, validation_set, graph_st_gen = build_datasets(config, 
                                              node_feat_names, 
                                              edge_feat_names, 
                                              node_label_name)
first_batch_graphs = training_batch_set[0]

node_feature_dim = first_batch_graphs.node_features.shape[-1]
edge_feature_dim = first_batch_graphs.edge_features.shape[-1]
label_dim = first_batch_graphs.node_labels.shape[-1]
node_feature_dim, edge_feature_dim, label_dim

(2, 2, 1)

In [25]:
print('Num of training batches: {}'.format(len(training_batch_set)))
print('Num of training batches: {}'.format(len(validation_set)))
print('Num of graphs in one training batch: {}'.format(training_batch_set[0].n_graphs))
print('Num of graphs in one validation batch: {}'.format(validation_set[0].n_graphs))

print('Node data shape in Train: {}'.format(training_batch_set[0].node_features.shape))
print('Node label data shape in Train: {}'.format(training_batch_set[0].node_labels.shape))
print('Edge data shape in Train: {}'.format(training_batch_set[0].edge_features.shape))

Num of training batches: 2
Num of training batches: 5
Num of graphs in one training batch: 32
Num of graphs in one validation batch: 5
Node data shape in Train: (832, 14, 2)
Node label data shape in Train: (832, 1, 1)
Edge data shape in Train: (7240, 14, 2)


## Run training loops

In [26]:
# del model, placeholders, tensors
epochs=10
first_batch_graphs = training_batch_set[0]

node_feature_dim = first_batch_graphs.node_features.shape[-1]
edge_feature_dim = first_batch_graphs.edge_features.shape[-1]
label_dim = first_batch_graphs.node_labels.shape[-1]
node_feature_dim, edge_feature_dim, label_dim

tensors, placeholders, model = build_model(
    config, node_feature_dim, edge_feature_dim, label_dim)

t_start = time.time()

init_ops = (tf.global_variables_initializer(),
            tf.local_variables_initializer())

# If there is already a session instance, close it and start a new one
if 'sess' in globals():
  sess.close()

sess = tf.Session()
sess.run(init_ops)

print('Done with initialization.')

accumulated_metrics = collections.defaultdict(list)

early_stop_last_n = 10
val_loss_list = []
training_done_flag = False
for i_epoch in range(epochs):
  for i_iter, batch in enumerate(training_batch_set):
    _, train_metrics = sess.run(
        [tensors['train_step'], tensors['metrics']['training']],
        feed_dict=fill_feed_dict(placeholders, batch))

    # Accumulate over minibatches to reduce variance in the training metrics
    for k, v in train_metrics.items():
      accumulated_metrics[k].append(v)

    if (i_iter + 1) % config['training']['print_after'] == 0:
      metrics_to_print = {
          k: np.mean(v) for k, v in accumulated_metrics.items()}
      info_str = ', '.join(
          ['%s %.4f' % (k, v) for k, v in metrics_to_print.items()])
      # reset the metrics
      accumulated_metrics = collections.defaultdict(list)

      if ((i_iter + 1) // config['training']['print_after'] %
          config['training']['eval_after'] == 0):
        eval_metrics = evaluate(
            sess, tensors['metrics']['validation'], placeholders,
            validation_set, config['evaluation']['batch_size'])
        info_str += ', ' + ', '.join(
            ['%s %.4f' % ('val/' + k, v) for k, v in eval_metrics.items()])
        val_loss_list.append(eval_metrics['mse_loss'])

      print('epoch %d, iter %d, %s, time %.2fs' % (
          i_epoch + 1, i_iter + 1, info_str, time.time() - t_start))
      t_start = time.time()


(2, 2, 1)

Done with encoder.
Done with GraphSTNet.
Model_inputs: {'node_features': <tf.Tensor 'Placeholder:0' shape=(?, 14, 2) dtype=float32>, 'edge_features': <tf.Tensor 'Placeholder_1:0' shape=(?, 14, 2) dtype=float32>, 'from_idx': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int32>, 'to_idx': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=int32>, 'graph_idx': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=int32>, 'n_graphs': 32}



Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting spars

Done with optimizer and model.


(None, None)

Done with initialization.
epoch 1, iter 1, loss 2.2284, grad_scale 3.5415, param_scale 15.7744, val/mse_loss 1.9194, time 3.77s
epoch 1, iter 2, loss 2.0003, grad_scale 2.0718, param_scale 15.7968, val/mse_loss 1.8198, time 0.90s
epoch 2, iter 1, loss 2.2030, grad_scale 0.2162, param_scale 15.8158, val/mse_loss 1.8227, time 1.68s
epoch 2, iter 2, loss 1.8865, grad_scale 0.2048, param_scale 15.8420, val/mse_loss 1.8210, time 0.89s
epoch 3, iter 1, loss 2.2036, grad_scale 0.1722, param_scale 15.8724, val/mse_loss 1.8201, time 1.67s
epoch 3, iter 2, loss 1.8844, grad_scale 0.0506, param_scale 15.9053, val/mse_loss 1.8201, time 0.92s
epoch 4, iter 1, loss 2.2015, grad_scale 0.0160, param_scale 15.9392, val/mse_loss 1.8202, time 1.68s
epoch 4, iter 2, loss 1.8850, grad_scale 0.1001, param_scale 15.9735, val/mse_loss 1.8200, time 0.91s
epoch 5, iter 1, loss 2.2011, grad_scale 0.0275, param_scale 16.0072, val/mse_loss 1.8200, time 1.70s
epoch 5, iter 2, loss 1.8839, grad_scale 0.0386, param_s

# Check model predictions

## For Spatio-temporal dataset

In [27]:

model_inputs = placeholders.copy()
del model_inputs['labels']

# build dict of lists
# day_when_to_make_pred | day_pred | node_name| pred | true
from collections import defaultdict

res = defaultdict(list)
total_nodes=26
horizon = config['data_param']['horizon']
for iter, val in enumerate(validation_set):
  n_graphs = val.n_graphs
  print('Eval on iter {}, total graph={}, total nodes={}'.format(iter, n_graphs, len(val.graph_idx)))
  
  y= model(n_graphs=n_graphs, **model_inputs)
  check_mse_loss = compute_loss(y, placeholders['labels'])  
  yhat, mse_loss_val = sess.run([y, check_mse_loss],
                                feed_dict=fill_feed_dict(placeholders, val))
  
  for i, day in enumerate(val.graph_idx):
    for j_pred in range(1): # we are only predicting for one step
      res['day_when_to_make_pred'].append(day)
      res['node_name'].append(node_idx2name[i%total_nodes])
      res['day_pred'].append(day+horizon+j_pred+1)   # should be on the day we are predicting for
      res['true_scaled'].append(val.node_labels[i][j_pred][0])
      res['pred_scaled'].append(yhat[i][j_pred])
res_df = pd.DataFrame(res)

Eval on iter 0, total graph=5, total nodes=130
Eval on iter 1, total graph=5, total nodes=130
Eval on iter 2, total graph=5, total nodes=130
Eval on iter 3, total graph=5, total nodes=130
Eval on iter 4, total graph=5, total nodes=130


In [28]:
res_df.head()

,day_when_to_make_pred,node_name,day_pred,true_scaled,pred_scaled
0,119,a,127,-2.195130,0.039446
1,119,b,127,1.446449,-0.061275
2,119,c,127,1.219629,-0.086640
3,119,d,127,-0.443102,-0.096840
4,119,e,127,1.045137,-0.110327


## Scale back the predictions and compute metrics


In [29]:
import sklearn, scipy, math
def compute_rmse(actual, predicted):
  mse = sklearn.metrics.mean_squared_error(actual, predicted)
  return math.sqrt(mse)

def compute_corr(actual, predicted):
  return scipy.stats.pearsonr(actual, predicted)[0]

def compute_mape(actual, predicted):
  y_true, y_pred = np.array(actual)+1, np.array(predicted)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100  
  # return sklearn.metrics.mean_absolute_percentage_error(actual, predicted)

def compute_rmsle(actual, predicted):
  assert len(actual) == len(predicted)
  return np.sqrt(np.mean((np.log(1+predicted) - np.log(1+actual))**2))

horizon = config['data_param']['horizon']
max_day = config['evaluation']['max_day']
n_step_ahead = 1
label_name_val = 'node_feat1_delta'

node_data_dict = graph_st_gen._node_data_dict
node_data_scaling_metrics = graph_st_gen._node_data_scaling_metrics
res_metrics = defaultdict(list)
all_regions_data = None 
for region in node_data_dict.keys():
  region_data = res_df.loc[(res_df['node_name']==region) & (res_df['day_when_to_make_pred']+horizon+1==res_df['day_pred'])].copy()
  region_data.set_index('day_pred', inplace=True)
  min_val, max_val =  node_data_scaling_metrics[region+label_name_val]

  # new cases per capital
  region_data['true'] = region_data['true_scaled']*(max_val-min_val)+min_val
  region_data['pred'] = region_data['pred_scaled']*(max_val-min_val)+min_val

  # region_data = region_data.loc[region_data['day_when_to_make_pred']<=241-horizon].copy()

  def cap_at_zero(val):
    if val<0:
      return 0.0
    else:
      return val

  region_data['pred'] = region_data.apply(lambda row: cap_at_zero(row['pred']), axis=1)
  
  data_to_save = region_data[['node_name', "day_when_to_make_pred", 'true', 'pred']].copy()
  data_to_save['horizon'] = horizon
  data_to_save['method'] ='spatiotemporal_gnn'
  data_to_save = data_to_save.rename(columns={"day_when_to_make_pred": "day", 
                                              "node_name": "state", 
                                              "pred": "pred",
                                              "horizon": "horizon",
                                              "true": "gt", 
                                              'method': "method"}, inplace=False)
  
  if all_regions_data is None:
    all_regions_data = data_to_save.copy()
  else:
    all_regions_data = all_regions_data.append(data_to_save, ignore_index=True)

  rmse_metric = compute_rmse(region_data['true'], region_data['pred'])
  res_metrics['region'].append(region)
  res_metrics['rmse'].append(rmse_metric)
  res_metrics['corr'].append(compute_corr(region_data['true'], region_data['pred']))
  res_metrics['mape'].append(compute_mape(region_data['true'], region_data['pred']))
  res_metrics['rmsle'].append(compute_rmsle(region_data['true'], region_data['pred']))

res_metrics_df = pd.DataFrame(res_metrics)

res_metrics_df.mean()

import datetime
all_regions_data['date'] = all_regions_data.apply(lambda row: (datetime.date(2020, 1, 1) + datetime.timedelta(row.day+horizon)).strftime('%Y-%m-%d'),axis=1)
all_regions_data.tail()


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/pyth

rmse       8.752330
corr            NaN
mape     100.000000
rmsle      1.725182
dtype: float64

,state,day,gt,pred,horizon,method,date
645,z,139,-10.659490,0.0,7,spatiotemporal_gnn,2020-05-26
646,z,140,4.449858,0.0,7,spatiotemporal_gnn,2020-05-27
647,z,141,-7.771705,0.0,7,spatiotemporal_gnn,2020-05-28
648,z,142,-10.059340,0.0,7,spatiotemporal_gnn,2020-05-29
649,z,143,-1.487060,0.0,7,spatiotemporal_gnn,2020-05-30


In [30]:
def plot_region(res_df, region, n_day_ahead=7, scaled=False, ax=None):
  col_to_view = ['true_scaled','pred_scaled']
  region_data = res_df.loc[(res_df['node_name']==region) & (res_df['day_when_to_make_pred']+n_day_ahead+1==res_df['day_pred'])].copy()
  region_data.set_index('day_pred', inplace=True)

  if scaled==False:
    col_to_view = ['true', 'pred']
    min_val, max_val =  node_data_scaling_metrics[region+label_name_val]
    region_data['true'] = region_data['true_scaled']*(max_val-min_val)+min_val
    region_data['pred'] = region_data['pred_scaled']*(max_val-min_val)+min_val

  region_data
  if ax is None:  
    plt.plot(region_data[col_to_view], title=region)
  region_data[col_to_view].plot(ax=ax)
  ax.set_title(region)
  ax.legend()

In [ ]:
m, n = 2, 3
fig, ax = plt.subplots(m, n, figsize=(15, 11))
regions = ['a','b','c','d','e','f']
for i in range(len(regions)):
  fx, fy = int(i/n), i%n
  ax[fx][fy]
  plot_region(res_df, regions[i], n_day_ahead=7, scaled=False, ax=ax[fx][fy])
